# Initialization

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from pathlib import Path
import os

# Add the project root to the Python path
BASE_DIR = Path(os.getcwd()).resolve().parent
if BASE_DIR not in sys.path:
    sys.path.append(str(BASE_DIR))

# Now you can use absolute imports
from post_processing.plotting_results_EMA import *

matplotlib.use('TkAgg')

SIM_NAME = "ema_road_model_08_05_2024"
TRAFFIC_TYPE = "passenger"  # "cargo", "passenger" or "combined"
# Load the data
PLOT_DIR = BASE_DIR / "plots"
DATA_DIR = (
    BASE_DIR.parent / "output_simulations" / SIM_NAME
)  # Folder with the results of the simulations as CSV files
DATA_SUBDIR_BRIDGES = (
    "bridges_ICratio"  # Subdirectory of the data directory where the results are stored
)
DATA_SUBDIR_ROAD_NETWORK = (
    f"road_network"  # Subdirectory of the data directory where the results are stored
)



### Load data

In [6]:
class Data:
    def __init__(self, filename):
        self.filename = filename
        self.data = load_results_single_df(self.filename)
        self.relative_data = self.relative_change()
        self.relative_data_transposed = self.relative_data.transpose()
        self.stats = self.get_stats()
    
    def relative_change(self):
        """
        Calculate the relative change to the previous row for all columns in the DataFrame
        """
        relative_df = self.data.pct_change() * 100
        return relative_df
    
    def get_stats(self, property="year"):
        if property not in ["year", "scenario"]:
            raise ValueError("The property should be either 'year' or 'scenario'")
        if property == "year":
            stats = self.relative_data_transposed.describe()
        elif property == "scenario":
            stats = self.relative_data.describe()
        print(stats)
        return stats

filename = f"{TRAFFIC_TYPE}_vkm.csv"
passenger_data = Data(filename)
pass



       2019        2020        2021        2022        2023        2024  \
count   0.0  500.000000  500.000000  500.000000  500.000000  500.000000   
mean    NaN    2.278143    1.941758    2.032325    2.536420    2.350439   
std     NaN    0.927360    0.790180    0.797133    0.681614    0.504046   
min     NaN   -0.441221   -0.192309   -0.096683    0.651068    0.052587   
25%     NaN    1.569061    1.448939    1.458176    2.114452    2.126832   
50%     NaN    2.292717    1.904709    2.090072    2.507660    2.476290   
75%     NaN    2.966593    2.498599    2.603983    2.946630    2.680720   
max     NaN    4.661162    4.466078    4.230036    4.559970    3.559078   

             2025        2026        2027        2028  ...        2041  \
count  500.000000  500.000000  500.000000  500.000000  ...  500.000000   
mean     2.397489    1.630635    2.414065    2.121483  ...    0.094018   
std      1.018830    0.521783    0.416745    0.562266  ...    0.784083   
min      0.384740    0.40019

## Relative change

In [4]:
# Filter scenarios where the value in 2036 increases and in 2037 decreases
def filter_scenarios(data, year1, year2, increase_threshold=4, decrease_threshold=0):
    filtered_scenarios = []
    for scenario in data.relative_data.columns:
        if data.relative_data.loc[year1, scenario] > increase_threshold and data.relative_data.loc[year2, scenario] < decrease_threshold:
            filtered_scenarios.append(scenario)
    return filtered_scenarios

scenarios = filter_scenarios(passenger_data, 2036, 2037)
scenario_numbers = [int(scenario.split("_")[-1]) for scenario in scenarios]
print("Scenarios where the value in 2036 increases and in 2037 decreases:", scenario_numbers)
print("Number of scenarios:", len(scenarios))

Scenarios where the value in 2036 increases and in 2037 decreases: [94, 101, 118, 140, 141, 156, 171, 175, 184, 186, 192, 217, 264, 326, 331, 362, 377, 460, 473, 480, 481, 494, 496, 500, 503, 544, 545]
Number of scenarios: 27


In [5]:
subset_2036_peak = passenger_data.data[scenarios]
subset_remaining = passenger_data.data.drop(columns=scenarios)
save_fig = False

# Check input types
if not isinstance(passenger_data.data, pd.DataFrame):
    raise TypeError("df_results must be a pandas DataFrame")
if not isinstance(save_fig, bool):
    raise TypeError("save_fig must be a boolean")

# Plot the results
fig, (ax1, ax2) = plt.subplots(
    1, 2, gridspec_kw={"width_ratios": [9, 1]}, sharey=True
)

# Draw the first subplot
ax1 = plt.subplot(1, 2, 1)
for scenario in subset_2036_peak.columns:
    ax1.plot(subset_2036_peak.index, subset_2036_peak[scenario], label=scenario, color="red", alpha=0.5, zorder=10)
for scenario in subset_remaining.columns:
    ax1.plot(subset_remaining.index, subset_remaining[scenario], label=scenario, color="gray", alpha=0.5, zorder=5)
ax1.set_ylim(0, passenger_data.data.max().max())
ax1.minorticks_on()
ax1.grid(which="minor", linestyle=":", linewidth="0.5", color="gray")
ax1.set_xlim(passenger_data.data.index.min(), passenger_data.data.index.max())
ax1.set_xlabel("Year")
ax1.set_ylabel(f"{TRAFFIC_TYPE.capitalize()} vehicle kilometers (vkm)")
ax1.set_title(
    f"Scenario Ensemble for {TRAFFIC_TYPE} vehicle kilometers (vkm). Simulation: {SIM_NAME}"
)
ax1.grid(True)
ax1.axvline(
    x=passenger_data.data.index[0], color="gray", linestyle="--", alpha=0.5
)  # Add vertical gridline at the first value

# Add a second subplot for KDE
ax2 = plt.subplot(1, 2, 2)
values_2030 = passenger_data.data.iloc[11]
values_2050 = passenger_data.data.iloc[-1]
sns.kdeplot(y=values_2030, fill=True)
sns.kdeplot(y=values_2050, fill=True)
ax2.legend(["2030", "2050"])
ax2.set_xlabel("Frequency")
ax2.set_title("Kernel density estimation")
ax2.grid(True)

if save_fig:
    plt.savefig(PLOT_DIR / f"{TRAFFIC_TYPE}_VKM.png")
plt.tight_layout()
plt.subplots_adjust(wspace=0.1)  # Adjust the spacing between subplots
plt.show()


